In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from keras.optimizers import SGD
import random
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

# New Section

In [ ]:
import json
with open('wearedone.json','rb') as json_data:
    intents = json.load(json_data)


In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])



In [ ]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)




2463 documents
409 classes ['', ' books_computer_engineering ', 'ACM_committee', 'Alumnis of RAIT', 'Auditorium', 'Books_Computer_Engineering', 'Books_EXTC', 'Books_Electronic', 'CSI_committee', 'Chancellor', 'Comps HOD', 'ETRX HOD', 'EXTC HOD', 'Events', 'Expert_Talk', 'Faculty', 'Founder', 'HOD', 'History of RAIT', 'Hostel', 'Hostel_Facilities', 'IEEE_committee', 'IIC_RAIT', 'ISA_ સમિતિ', 'ISA_Committee', 'ISA_committee', 'IT HOD', 'Instru HOD', 'International_Conferences', 'International_Papers_Computer_Engineering', 'International_Papers_EXTC', 'International_Papers_Electronic', 'International_Papers_Enginnering_Science', 'International_Papers_IT', 'International_Papers_Instrumentation', 'Kalaraag_committee', 'LMS', 'Lab', 'Library', 'Library_Info', 'National_Confereneces', 'Office', 'Officeફિસ', 'PCR_committee', 'PCR_委員会', 'PG', 'Patents', 'PhD', 'Principal', 'Query_for_PG_PhD', 'RAIT_Pay', 'RAITの卒業生', 'RAITの履歴', 'Railway_Concession_Form', 'TPC_committee', 'Timing', 'UG', 'UG_CS',

In [ ]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])


In [ ]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")




Training data created


In [ ]:
model = Sequential()
model.add(Dense(64, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(258, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(356, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=350, batch_size=65, verbose=1, validation_split=0.2)
#model.save('/contents/chatbot_model.h5', model)

print("model created")


Epoch 1/350
31/31 [==============================] - 0s 11ms/step - loss: 6.0044 - accuracy: 0.0046 - val_loss: 5.9862 - val_accuracy: 0.0183
Epoch 2/350
31/31 [==============================] - 0s 7ms/step - loss: 5.8796 - accuracy: 0.0178 - val_loss: 5.8643 - val_accuracy: 0.0061
Epoch 3/350
31/31 [==============================] - 0s 7ms/step - loss: 5.7015 - accuracy: 0.0183 - val_loss: 5.7644 - val_accuracy: 0.0203
Epoch 4/350
31/31 [==============================] - 0s 7ms/step - loss: 5.5470 - accuracy: 0.0299 - val_loss: 5.6232 - val_accuracy: 0.0345
Epoch 5/350
31/31 [==============================] - 0s 7ms/step - loss: 5.3830 - accuracy: 0.0411 - val_loss: 5.4655 - val_accuracy: 0.0568
Epoch 6/350
31/31 [==============================] - 0s 7ms/step - loss: 5.2451 - accuracy: 0.0487 - val_loss: 5.3306 - val_accuracy: 0.0771
Epoch 7/350
31/31 [==============================] - 0s 7ms/step - loss: 5.0643 - accuracy: 0.0599 - val_loss: 5.1700 - val_accuracy: 0.1136
Epoch 8/350


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
import json
import random
intents = json.loads(open('wearedone.json','rb').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


In [ ]:


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res




In [ ]:
chatbot_response('rait ची काही कॅन्टीन आहे?')

"होय आरएआयटीचे कॅनटीन नाव 'गार्डनर्स कॅफेऑन ग्राऊंड फ्लोर' आहे, स्वस्त किंमतीत अन्नाची गुणवत्ता उत्कृष्ट आहे आणि जवळजवळ सर्व प्रकारची पाककृती वेज आणि नॉन-वेजमध्ये उपलब्ध आहे"

In [ ]:
chatbot_response('कॉलेज कुठे आहे')

'सेक्टर 7, नेरूळ, नवी मुंबई, महाराष्ट्र 400706'

In [ ]:
chatbot_response('सर्वोच्च स्थान')

'सेक्टर 7, नेरूळ, नवी मुंबई, महाराष्ट्र 400706'

In [ ]:
chatbot_response('ਫੈਕਲਟੀ ਕਿਵੇਂ ਹੈ')

'1.rait ਸ਼ਖਸੀਅਤ ਦੇ ਵਿਕਾਸ ਦੇ ਸਾਰੇ ਪਹਿਲੂਆਂ ਨੂੰ ਪੈਦਾ ਕਰਨ ਦੀ ਆਗਿਆ ਦਿੰਦਾ ਹੈ ra n2. ਏਆਰਆਈਏਟੀ ਉੱਚ ਯੋਗਤਾ ਪ੍ਰਾਪਤ ਸਟਾਫ ਹੈ ਜੋ ਅਕਾਦਮਿਕ ਅਤੇ ਤਕਨੀਕੀ ਦੋਵਾਂ ਹੁਨਰਾਂ ਨੂੰ ਵਧਾਉਣ ਵਿਚ ਮਦਦਗਾਰ ਹੈ \n3. ਵੱਖ ਵੱਖ ਕਮਿitiesਨਿਟੀਜ਼ ਦੀ ਯੋਗਤਾ ਪ੍ਰਤਿਭਾਵਾਂ ਦੀ ਪੜਚੋਲ ਕਰਨ ਅਤੇ ਸਵੈ-ਵਿਸ਼ਵਾਸ ਵਧਾਉਣ ਵਿਚ ਸਹਾਇਤਾ ਕਰਦੀ ਹੈ'

In [ ]:
chatbot_response('ਹਾਇ')

'ਹੈਲੋ, ਪੁੱਛਣ ਲਈ ਧੰਨਵਾਦ'

In [ ]:
chatbot_response('कंप्यूटर इंजीनियरिंग')

' कंप्यूटर विज्ञान और व्यवसाय प्रणालियों में नेटवर्किंग और सूचना सुरक्षा में विशेषज्ञता के साथ कंप्यूटर इंजीनियरिंग। टीसीएस द्वारा डिजाइन कंप्यूटर पाठ्यक्रम'